# Практическое занятие №1

## Генерация случайных чисел и последовательностей

Подключаем необходимые либы

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Генерируем нормальное распределение

In [ ]:
mu, sigma = 0, 1 # mean and standard deviation
size = 10000
normal_distribution = np.random.normal(mu, sigma, size)

Проверим, что оно получилось с нужными нам параметрами

In [ ]:
# check mu
print(abs(mu - np.mean(normal_distribution)))

# check sigma
print(abs(sigma - np.std(normal_distribution)))

Построим гистограмму

In [ ]:
plt.hist(normal_distribution, 30, density=True)
plt.title('Normal distribution')
plt.show()

Генерируем равномерное распределение

In [ ]:
uniform_distribution = np.random.uniform(size=size)

Проверяем параметры, чему они будут равны?

https://www.statology.org/uniform-distribution/

In [ ]:
print(np.mean(uniform_distribution))

print(np.std(uniform_distribution))

Строим гистограмму, чтобы проверить

In [ ]:
plt.hist(uniform_distribution, 30, density=True)
plt.title('Uniform distribution')
plt.show()

Теперь проверим наши распределения при помощи теста Колмогорова-Смирнова

In [ ]:
from scipy import stats

In [ ]:
result = stats.ks_2samp(normal_distribution, uniform_distribution)
pthr = 0.05

print(result.pvalue > pthr)

После чего проверим их с "эталонными"

In [ ]:
ks_uniform = stats.uniform.rvs(size=size)

result = stats.ks_2samp(uniform_distribution, ks_uniform)

print(result.pvalue > pthr)

Проверить тоже самое для нормального, будет ли работать и как исправить?

[Линейный конгруэнтный генератор](https://www.geeksforgeeks.org/linear-congruence-method-for-generating-pseudo-random-numbers/)

Какое распределение получаем, посмотреть, построить

In [ ]:
# Initialize
seed_num = 42
multiplier = 5**5
increment = 0
modulus = (2**26)
unit = 10000

lcg = []

num_base = seed_num
for i in range(unit):
    rd = (multiplier * num_base + increment) % modulus
    lcg.append(rd)
    num_base = rd

lcg = lcg / np.max(lcg)


Теперь попробуем текущие распределение превратить в нормальное

Для этого будем использовать преобразование Бокса-Мюллера

In [ ]:
R = np.sqrt(-2 * np.log(lcg))
Theta = 2 * np.pi * lcg
Nm = R * np.cos(Theta)

И точно также, в экспоненциальное

In [ ]:
lmbd = 0.4
Em = (-1/lmbd)*np.log(lcg)

## Работа с изображениями

Считаем изображение и попробуем его отобразить

In [ ]:
import cv2

In [ ]:
img = cv2.imread('Mandrill.png')

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# cv2.imshow('img', img)
plt.imshow(img)

Дальше работать будем с серым (одноканальным) изображением

Точнее, будем проверять 3-канальное ли изображение и конвертировать его

In [ ]:
if len(img.shape) > 2:
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

plt.imshow(img, cmap='gray')
# cv2.imwrite('img.png', img)

Построим также гистограмму распределения яркости на изображении

In [ ]:
plt.hist(img.ravel(), 256, [0, 256])
plt.show()

Теперь добавим шума

Сначала Гауссов (нормальный)

In [ ]:
gaussian_noise = np.zeros(img.shape,dtype=np.uint8)
cv2.randn(gaussian_noise,128,20)
gaussian_noise = (gaussian_noise).astype(np.uint8)
gaussian_noise_img = cv2.add(img, gaussian_noise)

Посмотрим что, из себя представляет шум, и как выглядит зашумленное изображение и его гистограмма

Посчитаем [отношение Сигнал-Шум](https://ru.wikipedia.org/wiki/%D0%9F%D0%B8%D0%BA%D0%BE%D0%B2%D0%BE%D0%B5_%D0%BE%D1%82%D0%BD%D0%BE%D1%88%D0%B5%D0%BD%D0%B8%D0%B5_%D1%81%D0%B8%D0%B3%D0%BD%D0%B0%D0%BB%D0%B0_%D0%BA_%D1%88%D1%83%D0%BC%D1%83)

In [ ]:
mse = np.mean((img - gaussian_noise_img) ** 2)
max_pixel = 255.0
psnr = 20 * np.log10(max_pixel/ np.sqrt(mse))

Теперь попробуем добавить этот же Гауссов шум другим способом, как мы делали раньше

In [ ]:
normal_distribution = np.random.normal(0, 1, (img.shape))

# plt.hist(normal_distribution.ravel(), 256, density=True)
# plt.title('Normal distribution')
# plt.show()

normal_distribution_img = img + normal_distribution.astype(np.uint8)

# plt.imshow(normal_distribution_img, cmap='gray')


Добавим шум не только аддитивно, но мультипликативно

Затем, проделаем тоже самое с другими шумами (равномерным и экспоненциальным)

И пытаемся добавить импульсный шум (он же salt-and-pepper)

In [ ]:
noise_density = 10 # процент зашумления

sp_img = img.copy()

for _ in range(round((noise_density * img.shape[0] * img.shape[1])/ 100)):
    x = np.random.randint(img.shape[1])
    y = np.random.randint(img.shape[0])

    sp_img[x, y] = (np.random.uniform(0, 1) > 0.5) * 255

plt.imshow(sp_img, cmap='gray')